In [ ]:

#r "nuget:Microsoft.DotNet.Interactive.SqlServer,1.0.0-beta.22175.2"
#r "nuget:Microsoft.DotNet.Interactive,1.0.0-beta.22175.2"
#r "nuget:Microsoft.DotNet.Interactive.PowerShell,1.0.0-beta.22175.2"
#r "nuget:Microsoft.DotNet.Interactive.Http,1.0.0-beta.22175.2"

#r "nuget:FSharp.Core, *-*"
#r "nuget:Plotly.NET,*-*"
#r "nuget: Plotly.NET.Interactive, *-*"
using Plotly.NET;




In [ ]:
#!connect mssql --kernel-name splSnapshots "Server=localhost;Database=SeattlePublicLibraryOpenData;Trusted_Connection=True;TrustServerCertificate=true;"

In [ ]:
#!sql-splSnapshots --name itemTypeAgeGroup


;WITH ageGroup_cte
AS (SELECT
  ReportDate,
  AgeGroup,
  COUNT(DISTINCT bibnum) AS ItemCount,
  SUM(ei.itemcount) AS InventoryCount

FROM ElementInventory ei
INNER JOIN ItemType it
  ON ei.ItemTypeCode = it.Code
WHERE ei.ItemCount > 0
GROUP BY ReportDate,
         AgeGroup),
calculatedReportDateTotal_cte
AS (SELECT
  ReportDate,
  SUM(itemcount) AS itemCountSum,
  SUM(inventoryCount) AS inventoryCountSum
FROM ageGroup_cte
GROUP BY ReportDate)
SELECT
  r.ReportDate,
  AgeGroup,
  CAST(ItemCount AS float) / itemCountSum * 100 AS AgeGroupItemPercentage,
  CAST(InventoryCount AS float) / inventoryCountSum * 100 AS AgeGroupInventoryPercentage

FROM calculatedReportDateTotal_cte r
INNER JOIN ageGroup_cte a
  ON a.ReportDate = r.ReportDate

ORDER BY ReportDate, AgeGroup




In [ ]:
using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;



Trace t = new Trace("bar");
t.SetValue("x", new string[][] { new string[]{"a","a","b","b"}, new string[]{"sub1", "sub2","sub1", "sub2"} });
t.SetValue("y", new int[]{2,4,6,8});
//t.SetValue("x", new int[][] { new int[]{1}, new int[]{2} });
t.SetValue("name", "name1");

Trace t2 = new Trace("bar");
t2.SetValue("x", new string[][] { new string[]{"a","a","b","b"}, new string[]{"sub1", "sub2","sub1", "sub2"} });
t2.SetValue("y", new int[]{1,2,3,4});
//t.SetValue("x", new int[][] { new int[]{1}, new int[]{2} });
t2.SetValue("name", "name2");
//var tra = Trace2D.initBar(FSharpFunc<Trace2D, Trace2D>.FromConverter(new Converter<Trace2D, Trace2D>(x => x)));
var charts = new GenericChart.GenericChart[] {GenericChart.ofTraceObject(false, t), GenericChart.ofTraceObject(false,t2)};
GenericChart.combine(charts)
//.WithLayout(Layout.init<int>(BarMode:StyleParam.BarMode.Stack))


<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#r "nuget:Plotly.NET,*-*"
#r "nuget: Plotly.NET.Interactive,*-*"

open Plotly.NET

let trace x y name =  //Workaround
    let tmp = Trace("bar")
    tmp?x <- x
    tmp?y <- y
    tmp?name <- name
    tmp
[
    GenericChart.ofTraceObject false (trace [["First"; "First";"Second";"Second"];["A"; "B"; "A"; "B"]] [2;3;1;5] "Adults")
    GenericChart.ofTraceObject false (trace [["First"; "First";"Second";"Second"];["A"; "B"; "A"; "B"]] [8;3;6;5] "Children")
]
|> Chart.combine
|> Chart.withLayout(Layout.init(Title = Title.init("Multi-category axis"), Width = 700))

Installed Packages Plotly.NET, 2.0.0 Plotly.NET.Interactive, 2.0.0

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#!share --from sql-splSnapshots itemTypeAgeGroup

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;

string[] years = (itemTypeAgeGroup[0].Data.GroupBy(x=>((DateTime)x["ReportDate"]).Year).OrderBy(x=>x.Key.ToString())
.SelectMany(x=>new string[]{x.Key.ToString(), x.Key.ToString()})).ToArray();

string[] inventoryCategory = (Enumerable.Repeat(new string[]{"Item Percentage", "Inventory Percentage"}, years.Length/2).SelectMany(x=>x)).ToArray();
List<Trace> traces = new();

foreach(var g in itemTypeAgeGroup[0].Data.GroupBy(x=> x["AgeGroup"]).OrderBy(x=>x.Key.ToString())){

    var itemPercentages = g.GroupBy(x=>((DateTime)x["ReportDate"]).Year).Select(x=> x.Average(x=>(double)x["AgeGroupItemPercentage"]));
    var inventoryPercentages = g.GroupBy(x=>((DateTime)x["ReportDate"]).Year).Select(x=> x.Average(x=>(double)x["AgeGroupInventoryPercentage"]));

    Trace t = new Trace("bar");
    t.SetValue("x", new string[][]{years, inventoryCategory} );
    t.SetValue("y", itemPercentages.Zip(inventoryPercentages, (x,y) => new double[]{x,y}).SelectMany(x=> x));
    t.SetValue("name", g.Key.ToString());
    traces.Add(t);

    
    // charts.Add(
    //     Chart2D.Chart.Column<string, double, string, string, string>(
    //         Name:g.Key.ToString()  ,
    //         keysValues: new List<Tuple<string,double>>(
    //             //ageGroups.First(x=>x.Key=="Adult").Select(kvp => new Tuple<string, double>(((DateTime)kvp["ReportDate"]).ToShortDateString(),(double) kvp["AgeGroupItemPercentage"]))
    //             g.GroupBy(x=>((DateTime)x["ReportDate"]).Year).Select(x=> new Tuple<string, double>(x.Key.ToString(), x.Average(x=>(double)x["AgeGroupItemPercentage"])))
    //         )//{new Tuple<string,double>(g.Key.Year.ToString(),g.Average(x=>(double)x["AgeGroupItemPercentage"]))}
            
    //     )
    //     .WithXAxisStyle(title: Title.init(),ShowGrid:false, ShowLine:true));
}
var layout = Layout.init<int>(
    Title: Title.init("Age Group Composition %"),
    Width:1500,
    DragMode:StyleParam.DragMode.Pan,
    BarMode:StyleParam.BarMode.Stack
    ,    Colorway: Color.fromColors(
        new List<Color>() { Color.fromKeyword(ColorKeyword.CornflowerBlue), Color.fromKeyword(ColorKeyword.SeaGreen) 
        , Color.fromKeyword(ColorKeyword.Pink), Color.fromKeyword(ColorKeyword.Yellow)})
);

#!html
<a id="ageGroup"></a>


#!csharp

GenericChart.combine(new GenericChart.GenericChart[]{GenericChart.ofTraceObject(true, traces[0]), GenericChart.ofTraceObject(true, traces[1])})
.WithLayout(layout)

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#!sql-splSnapshots --name itemTypeFormat

SELECT ReportDate,
       FormatGroup,
       Count(DISTINCT bibnum) as ItemCount,
       Sum(ei.itemcount) as InventoryCount
FROM   ElementInventory ei
       INNER JOIN ItemType it
               ON ei.ItemTypeCode = it.Code

GROUP  BY ReportDate,
          FormatGroup 

In [ ]:
#!share --from sql-splSnapshots itemTypeFormat

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;



In [ ]:
#!sql-splSnapshots --name itemCollectionFormat

-- SELECT ReportDate,
--        FormatGroup,
--        Count(DISTINCT bibnum) as ItemCount,
--        Sum(ei.itemcount) as InventoryCount
-- FROM   ElementInventory ei
--        INNER JOIN ItemCollection ic
--                ON ei.ItemCollectionCode = ic.Code
-- GROUP  BY ReportDate,
--           FormatGroup 